# Demo chatzilla

## Imports

first 7 lines are for relative imports using jupyter notebooks

In [ ]:
import sys
import os
# Add the parent directory to the path
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from dotenv import load_dotenv
from chatzilla import zillaping, PromptOllama, ChatOllama
from chatzilla.logger import save_history_to_json

In [2]:
load_dotenv() # rename .env.sample to .env
ollama_url_ping = os.getenv("OLLAMA_URL") # http://localhost:11434
ollama_url_prompt = os.getenv("OLLAMA_GEN") # http://localhost:11434/api/generate
ollama_url_chat = os.getenv("OLLAMA_CHAT") # http://localhost:11434/api/chat
model = os.getenv("DEFAULT_MODEL") # llama3.1

## Ping ollama

In [3]:
zillaping(ollama_url_ping)

'Ollama is running'

## Prompt

Single prompt without any chat history

In [4]:
prompt = "tell me a bill burr joke"
joke = PromptOllama(prompt, model, ollama_url_prompt)
print(joke)

Here's one:

"You ever notice that anyone driving slower than you is an idiot, and anyone driving faster than you is a maniac? That's not just a coincidence, folks. That's a universal truth." - Bill Burr


## Chat with history

In [ ]:
chat = ChatOllama(ollama_url_chat, model)
msg1 = chat.begin(prompt)
msg2 = chat.next("make the joke edgy")

print(f"first message received:\n\t{msg1}\n\n")
print(f"second message received:\n\t{msg2}")

first message received:

Here's one:

"You know what's wild? We spend the first year of a child's life teachin' 'em to walk and talk, and the rest of their lives tellin' 'em to shut up and sit down. It's like, what are we even doin'? It's like, I'm not sayin' my kid's stupid or nothin', but I had to explain to him why we can't just take a 30-minute nap on the side of the road every time he gets tired."
second message received:


"Have you ever noticed that people who are always telling you to 'follow your heart' and 'be true to yourself'? Those guys are usually the ones who have never had a job, never paid a bill on time, and still live in their mom's basement at 32 years old... (pause) ...and I'm pretty sure they're also the same people who are always trying to get you to sign up for one of those 'multi-level marketing' schemes. You know, the ones where you pay $500 for a vacuum cleaner and then sell vacuum cleaners to your friends? Yeah, good luck with that, buddy... said no one ever

In [ ]:
save_history_to_json(chat.history())